In [1]:
import sys
sys.path.append("../")

In [2]:
import solipsism.core.lpml as lpml
from solipsism.tools import file_io
from solipsism.core.system import System
from solipsism.core.context import Context
from solipsism.core.llm import GeminiLLM

In [3]:
system = System()
llm = GeminiLLM()

In [4]:
system.add_tool(file_io.ReadFileTool(root_path="../"))
system.add_tool(file_io.WriteFileTool(root_path="../"))
system.add_tool(file_io.ListFilesTool(root_path="../"))

In [5]:
context = Context(llm, system, "../solipsism/prompts/root_prompt.lpml")

In [6]:
await context.start(initial_task="None", max_turns=20)

In [7]:
print(context._build_full_prompt())

<rule name="LPML">
All messages must be formatted in LPML (LLM-Prompting Markup Language) format. LPML element ::= <tag attribute="value">content</tag> or <tag/>.
Tags determine the meaning and function of the content. The content must not contradict the definition of the tag.
</rule>

<rule name="solipsism">
本プロジェクト solipsism は、AIアシスタントとシステムのペアを「コンテクスト」とする、コンテクストオーケストレーションシステムである。
Solipsism は様々なコンテクストを生成・管理する。コンテクストは協働して課題解決を行う。
</rule>

<rule name="context">
アシスタントはシステムとの対話ループ（=コンテクスト）を通じて、与えられた課題を解決しなければならない。
システムはアシスタントに対して各種機能を提供する。アシスタントはLPMLタグ使用により、システムの機能を操作できる。システムは同時に複数の機能を実行可能である。
システムはアシスタントの出力をパースし、タグを実行する。実行結果はシステムの非同期結果キューに返されるため、システムは一定時間間隔ごとにキューをポールし、結果があればアシスタントに伝える。
</rule>

<rule name="コンテクスト間メッセージング">
Sendタグが与えられている場合、アシスタントはこのタグを用いて他のコンテクストと通信可能である。
人間のユーザーも一種のコンテクストとして扱われることに注意する。
コンテクストは非同期に実行されるため、システムが他のコンテクストからの応答を受信するまでに時間がかかることがある。５ターンは待つこと。
</rule>

<rule name="ワークフロー">
1. コンテクストが生成されたら、アシスタントはまず read_file タグを用いて ./solipsism/notes/readme.md を読む。
2. システムとの対

In [8]:
context.state

<ContextState.WAITING: 3>

In [8]:
print(system.get_tool_definitions())

<define_tag name="read_file">
Reads the content of a specified file.
Attributes:
    - path (required): The path to the file.
    - line_numbers (optional): If "true", prepends line numbers to the output.
</define_tag>

<define_tag name="write_file">
Writes or modifies a file. The content is placed inside the tag.
Attributes:
    - path (required): The path to the file.
    - mode (optional): "overwrite", "append", "replace_lines", "insert_at_line". Default: "overwrite".
    - start_line, end_line (for replace_lines): The 1-indexed line range to replace.
    - line (for insert_at_line): The 1-indexed line number to insert at.
</define_tag>


In [4]:
print(file_io.ListFilesTool.tag_definition)


    <define_tag name="list_files">
    test
    </define_tag>
    


In [3]:
llm = GeminiLLM()

In [4]:
file = llm.upload_file("../solipsism/core/lpml.py")
a = await llm.generate(f"../solipsism/core/lpml.pyについて説明して")

In [5]:
print(a.text)

`../solipsism/core/lpml.py` ファイルは、LPML (Lightweight Python Markup Language) と呼ばれるシンプルなマークアップ言語を解析、逆解析、および操作するためのユーティリティを提供します。LPML は、HTML や XML に似ていますが、Python で簡単に処理できるように設計されています。

以下に、ファイルの各部分の詳細な説明を示します。

### インポート

```python
import re
from typing import List, Dict, Union, Optional
```

- `re`: 正規表現を操作するためのPythonの標準モジュールです。LPMLタグのパターンマッチングに使用されます。
- `typing`: 型ヒントを提供するためのモジュールです。コードの可読性と保守性を向上させます。

### 型定義

```python
Attributes = Dict[str, str]
Element = Dict[str, Union[str, Attributes, List['Element']]]
LPMLTree = List[Union[str, Element]]
```

- `Attributes`: キーが文字列で値も文字列である辞書型を表します。これはLPML要素の属性を格納するために使用されます。例: `{'id': 'my-id', 'class': 'my-class'}`。
- `Element`: LPML要素を表す辞書型です。
    - `'tag'`: 要素のタグ名 (文字列)。
    - `'attributes'`: 要素の属性 (Attributes型)。
    - `'content'`: 要素の内容。これは以下のいずれかになります。
        - `str`: 単一の文字列コンテンツ。
        - `List['Element']`: 子要素のリスト。
        - `None`: 空タグの場合 (例: `<br/>`)。
- `LPMLTree`: LPMLドキュメント全体を表すリスト型です。このリストには、文字列 (テキストコンテンツ) または `Element` オブジェクトが含まれ

In [5]:
file

NameError: name 'file' is not defined

In [6]:
a = await llm.generate("こんにちは!")

In [7]:
print(a.text)

`files/solipsism-core-lpml-py` は、LPML (Lightweight Python Markup Language) と呼ばれるマークアップ言語を解析 (parse) し、逆解析 (deparse) するためのPythonモジュールです。XML/HTMLに似た構造を持つテキストデータを、Pythonのデータ構造（辞書とリストのネスト）に変換したり、その逆を行ったりする機能を提供します。

以下に、このモジュールの主要な部分と機能について詳しく説明します。

---

### 1. LPMLのデータ構造の定義

このモジュールでは、LPMLのツリー構造を表現するために以下の型エイリアスが定義されています。

```python
Attributes = Dict[str, str]
Element = Dict[str, Union[str, Attributes, List['Element']]]
LPMLTree = List[Union[str, Element]]
```

- **`Attributes`**: タグの属性を表現する辞書。キーと値はどちらも文字列です (例: `{'id': 'my-div', 'class': 'container'}`)。
- **`Element`**: LPMLの要素（タグ）を表現する辞書。
    - `'tag'`: タグ名 (文字列)。
    - `'attributes'`: `Attributes` 型の辞書。
    - `'content'`: 要素の内容。これは以下のいずれかになります。
        - `None`: 空のタグ (例: `<br/>`)。
        - `str`: テキストコンテンツ。
        - `List['Element']`: ネストされたLPMLツリー。
- **`LPMLTree`**: LPMLドキュメント全体、または要素のコンテンツを表現するリスト。このリストには、テキストコンテンツ (文字列) または`Element`オブジェクトが含まれます。

---

### 2. 正規表現パターン

LPMLのタグを識別するために、いくつかの正規表現パターンが定義されています。

- **`PATTERN_ATTRIBUT

In [3]:
system = System()
system.add_tool(file_io.ListFilesTool(root_path="./"))
await system.process_llm_output('<list_files path="./"></list_files><list_files path="./"><a><</list_files>')


2

In [4]:
a = await system.get_tool_results_as_lpml()

In [5]:
print(a)

<output tool="list_files" path="./">
Untitled.ipynb
.ipynb_checkpoints
</output>

<output tool="list_files" path="./">
Untitled.ipynb
.ipynb_checkpoints
</output>


In [13]:
list({"a":1}.keys()) + [2]

['a', 2]

In [16]:
my_list = [1, 2, 3]
result = [item for sublist in [[x, ','] for x in my_list] for item in sublist][:-1]
result

[1, ',', 2, ',', 3]

In [17]:
sum([[x, ','] for x in my_list], [])

[1, ',', 2, ',', 3, ',']

In [6]:
lpml.deparse(lpml.parse("<hoge  />"))

'<hoge/>'

In [10]:
import asyncio
import random

# プロデューサーのコルーチン
async def producer(queue: asyncio.Queue, num_items: int):
    """
    指定された数のアイテムを生成し、キューに追加します。
    """
    for i in range(num_items):
        item = f"item-{i}"
        await asyncio.sleep(random.uniform(0.1, 0.5))  # データ生成のシミュレーション
        await queue.put(item)
        print(f"Producer put: {item}")
    await queue.put(None)  # 終了シグナルとしてNoneを追加

# コンシューマーのコルーチン
async def consumer(queue: asyncio.Queue):
    """
    キューからアイテムを取得し、処理します。
    """
    while True:
        item = await queue.get()
        if item is None:
            await queue.put(None)  # 他のコンシューマーに終了シグナルを転送
            break
        print(f"Consumer got: {item}")
        await asyncio.sleep(random.uniform(0.1, 0.5))  # データ処理のシミュレーション

# メインのコルーチン
async def main():
    queue = asyncio.Queue()
    num_items_to_produce = 5

    # プロデューサーとコンシューマーのタスクを同時に実行
    producer_task = asyncio.create_task(producer(queue, num_items_to_produce))
    consumer_task = asyncio.create_task(consumer(queue))

    # 両方のタスクが完了するのを待つ
    await asyncio.gather(producer_task, consumer_task)


In [12]:
await main()

Producer put: item-0
Consumer got: item-0
Producer put: item-1
Consumer got: item-1
Producer put: item-2
Consumer got: item-2
Producer put: item-3
Consumer got: item-3
Producer put: item-4
Consumer got: item-4


In [6]:
import asyncio

queue = asyncio.Queue()
file_io.ListFilesTool().run({"tag": "hoge", "attributes": {"path": "./"}}, queue)
await asyncio.gather(queue)

/tmp/ipykernel_121514/100004491.py:4: RuntimeWarning: coroutine 'ListFilesTool.run' was never awaited
  file_io.ListFilesTool().run({"tag": "hoge", "attributes": {"path": "./"}}, queue)


TypeError: An asyncio.Future, a coroutine or an awaitable is required

In [ ]:
Element(

In [11]:
lpml.Element(tag="tag")

TypeError: Type Dict cannot be instantiated; use dict() instead

In [ ]:

async def func():
    asyncio.sleep(3)
    return None

async for i in range(5):
    queue.put